# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 27 2022 9:52AM,238251,20,ALU0006940,Alumier Labs Inc.,Released
1,May 27 2022 9:52AM,238251,20,ALU0006941,Alumier Labs Inc.,Released
2,May 27 2022 9:52AM,238251,20,ALU0006942,Alumier Labs Inc.,Released
3,May 27 2022 9:52AM,238251,20,ALUR272368480,Alumier Labs Inc.,Released
4,May 27 2022 9:52AM,238251,20,ALUR975790131,Alumier Labs Inc.,Released
5,May 27 2022 9:52AM,238251,20,ALU0006943,Alumier Labs Inc.,Released
6,May 27 2022 9:52AM,238251,20,ALU0006944,Alumier Labs Inc.,Released
7,May 27 2022 9:52AM,238251,20,ALU0006945,Alumier Labs Inc.,Released
8,May 27 2022 9:52AM,238251,20,ALU0006946,Alumier Labs Inc.,Released
9,May 27 2022 9:52AM,238251,20,ALU0006947,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,238247,Released,31
33,238248,Released,38
34,238249,Released,1
35,238250,Released,20
36,238251,Released,29


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
238247,NaN,NaN,31.0
238248,NaN,NaN,38.0
238249,NaN,NaN,1.0
238250,NaN,NaN,20.0
238251,NaN,NaN,29.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
238159,0.0,0.0,1.0
238165,0.0,4.0,5.0
238168,0.0,0.0,1.0
238171,0.0,0.0,2.0
238177,0.0,0.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
238159,0.0,0,1
238165,0.0,4,5
238168,0.0,0,1
238171,0.0,0,2
238177,0.0,0,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,238159,0.0,0,1
1,238165,0.0,4,5
2,238168,0.0,0,1
3,238171,0.0,0,2
4,238177,0.0,0,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,238159,0.0,,1
1,238165,0.0,4,5
2,238168,0.0,,1
3,238171,0.0,,2
4,238177,0.0,,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 27 2022 9:52AM,238251,20,Alumier Labs Inc.
29,May 27 2022 9:45AM,238248,10,ISDIN Corporation
67,May 27 2022 9:43AM,238250,10,"Methapharm, Inc."
87,May 27 2022 9:38AM,238249,10,ISDIN Corporation
88,May 27 2022 9:37AM,238247,10,ISDIN Corporation
119,May 27 2022 9:16AM,238243,10,ISDIN Corporation
146,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC"
153,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated"
154,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC"
156,May 27 2022 8:46AM,238237,10,Beach Products Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,May 27 2022 9:52AM,238251,20,Alumier Labs Inc.,0.0,,29
1,May 27 2022 9:45AM,238248,10,ISDIN Corporation,0.0,,38
2,May 27 2022 9:43AM,238250,10,"Methapharm, Inc.",0.0,,20
3,May 27 2022 9:38AM,238249,10,ISDIN Corporation,0.0,,1
4,May 27 2022 9:37AM,238247,10,ISDIN Corporation,0.0,,31
5,May 27 2022 9:16AM,238243,10,ISDIN Corporation,0.0,,27
6,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC",0.0,6,1
7,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated",0.0,,1
8,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC",0.0,,2
9,May 27 2022 8:46AM,238237,10,Beach Products Inc,0.0,9,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 9:52AM,238251,20,Alumier Labs Inc.,29,
1,May 27 2022 9:45AM,238248,10,ISDIN Corporation,38,
2,May 27 2022 9:43AM,238250,10,"Methapharm, Inc.",20,
3,May 27 2022 9:38AM,238249,10,ISDIN Corporation,1,
4,May 27 2022 9:37AM,238247,10,ISDIN Corporation,31,
5,May 27 2022 9:16AM,238243,10,ISDIN Corporation,27,
6,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC",1,6
7,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated",1,
8,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC",2,
9,May 27 2022 8:46AM,238237,10,Beach Products Inc,6,9


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 9:52AM,238251,20,Alumier Labs Inc.,29,
1,May 27 2022 9:45AM,238248,10,ISDIN Corporation,38,
2,May 27 2022 9:43AM,238250,10,"Methapharm, Inc.",20,
3,May 27 2022 9:38AM,238249,10,ISDIN Corporation,1,
4,May 27 2022 9:37AM,238247,10,ISDIN Corporation,31,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 9:52AM,238251,20,Alumier Labs Inc.,29.0,NaN
1,May 27 2022 9:45AM,238248,10,ISDIN Corporation,38.0,NaN
2,May 27 2022 9:43AM,238250,10,"Methapharm, Inc.",20.0,NaN
3,May 27 2022 9:38AM,238249,10,ISDIN Corporation,1.0,NaN
4,May 27 2022 9:37AM,238247,10,ISDIN Corporation,31.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 9:52AM,238251,20,Alumier Labs Inc.,29.0,0.0
1,May 27 2022 9:45AM,238248,10,ISDIN Corporation,38.0,0.0
2,May 27 2022 9:43AM,238250,10,"Methapharm, Inc.",20.0,0.0
3,May 27 2022 9:38AM,238249,10,ISDIN Corporation,1.0,0.0
4,May 27 2022 9:37AM,238247,10,ISDIN Corporation,31.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3096855,136.0,9.0
12,238210,1.0,0.0
15,952901,19.0,21.0
16,952825,9.0,2.0
18,238159,1.0,0.0
19,714623,3.0,18.0
20,952785,42.0,5.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3096855,136.0,9.0
1,12,238210,1.0,0.0
2,15,952901,19.0,21.0
3,16,952825,9.0,2.0
4,18,238159,1.0,0.0
5,19,714623,3.0,18.0
6,20,952785,42.0,5.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,136.0,9.0
1,12,1.0,0.0
2,15,19.0,21.0
3,16,9.0,2.0
4,18,1.0,0.0
5,19,3.0,18.0
6,20,42.0,5.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,136.0
1,12,Released,1.0
2,15,Released,19.0
3,16,Released,9.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20
Status,,,,,,,
Executing,9.0,0.0,21.0,2.0,0.0,18.0,5.0
Released,136.0,1.0,19.0,9.0,1.0,3.0,42.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20
0,Executing,9.0,0.0,21.0,2.0,0.0,18.0,5.0
1,Released,136.0,1.0,19.0,9.0,1.0,3.0,42.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20
0,Executing,9.0,0.0,21.0,2.0,0.0,18.0,5.0
1,Released,136.0,1.0,19.0,9.0,1.0,3.0,42.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()